In [1]:
import pandas as pd
import numpy as np
import time
import os
import zipfile
import pandas_gbq
import glob
from google.cloud import bigquery
import re

In [2]:
client = bigquery.Client()

In [5]:
query = """ SELECT ANO as ano, MES as mes, TIPO_PENSAO as tipo_pensao, DATA_INICIO_PENSAO as data_inicio_pensao, 
            1 as total FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2024_abr_siape_pensionista_cadastro` 
        """  

In [6]:
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')
df

Downloading: 100%|██████████|


,ano,mes,tipo_pensao,data_inicio_pensao,total
0,2024,4,Pessoa designada,29/05/2002,1
1,2024,4,Pai / Mae,17/07/1995,1
2,2024,4,Pai / Mae,22/07/1993,1
3,2024,4,Pessoa designada,20/08/2003,1
4,2024,4,Pai / Mae,22/08/2003,1
...,...,...,...,...,...
283365,2024,4,(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a),09/02/2002,1
283366,2024,4,(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a),01/01/2002,1
283367,2024,4,(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a),29/07/2003,1
283368,2024,4,(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a),16/02/2008,1


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 283370 entries, 0 to 283369
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   ano                 283370 non-null  Int64 
 1   mes                 283370 non-null  Int64 
 2   tipo_pensao         283370 non-null  object
 3   data_inicio_pensao  282628 non-null  object
 4   total               283370 non-null  Int64 
dtypes: Int64(3), object(2)
memory usage: 11.6+ MB


In [8]:
pensoes = df['data_inicio_pensao'].isnull()
pensoes.value_counts()

False    282628
True        742
Name: data_inicio_pensao, dtype: int64

In [26]:
pensoes = df['data_inicio_pensao'].isnull()
pensoes.value_counts()

False    237551
True         42
Name: data_inicio_pensao, dtype: int64

In [9]:
df = df.dropna() # Foram dropadas 42 linhas, equivalente ao total de valores nulos indicados acima
df

,ano,mes,tipo_pensao,data_inicio_pensao,total
0,2024,4,Pessoa designada,29/05/2002,1
1,2024,4,Pai / Mae,17/07/1995,1
2,2024,4,Pai / Mae,22/07/1993,1
3,2024,4,Pessoa designada,20/08/2003,1
4,2024,4,Pai / Mae,22/08/2003,1
...,...,...,...,...,...
283365,2024,4,(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a),09/02/2002,1
283366,2024,4,(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a),01/01/2002,1
283367,2024,4,(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a),29/07/2003,1
283368,2024,4,(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a),16/02/2008,1


In [11]:
df['data_inicio_pensao'] = pd.to_datetime(df['data_inicio_pensao'], dayfirst=True, format= "%d/%m/%Y") #transformando em data

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_9064\3566975300.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['data_inicio_pensao'] = pd.to_datetime(df['data_inicio_pensao'], dayfirst=True, format= "%d/%m/%Y") #transformando em data


In [12]:
df['ano_inicio_pensao'] = df['data_inicio_pensao'].dt.year #pegando apenas o ano

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_9064\3811510978.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ano_inicio_pensao'] = df['data_inicio_pensao'].dt.year #pegando apenas o ano


In [13]:
df['ano_inicio_pensao'].unique()

array([2002, 1995, 1993, 2003, 2021, 1996, 2016, 2000, 1998, 2022, 2001,
       2004, 2019, 2017, 2013, 2020, 1997, 2023, 1992, 2014, 2015, 1999,
       2018, 2012, 2005, 2006, 2008, 2010, 2007, 2009, 1991, 1980, 1987,
       1984, 1988, 2011, 1994, 1981, 1971, 1990, 1973, 1985, 1970, 1979,
       1975, 1967, 1983, 1982, 1954, 1962, 1978, 1935, 1989, 1965, 1939,
       1986, 1974, 1966, 1968, 1969, 1976, 1977, 1958, 2024, 1959, 1951,
       1972, 1945, 1953, 1964, 1957, 1956, 1963, 1955, 1961, 1960, 1943,
       1952, 1949, 1947, 1950, 1942, 1937, 1944, 1948, 1933, 1932, 1926,
       1929, 1928, 1946, 1936, 1940, 1930, 1923, 1938, 1941, 1934, 1927,
       1931, 1925, 1921, 1924, 1922], dtype=int64)

In [14]:
limites = [0, 1929, 1939, 1949, 1959, 1969, 1979, 1989, 1999, 2009, 2019, 2024]
categorias = [1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020]

df['decada_inicio_pensao'] = pd.cut(df['ano_inicio_pensao'], bins=limites, labels=categorias)


C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_9064\2468368124.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['decada_inicio_pensao'] = pd.cut(df['ano_inicio_pensao'], bins=limites, labels=categorias)


In [15]:
df['tipo_pensao'].unique()

array(['Pessoa designada', 'Pai / Mae', 'Irmao (a)', 'Neto (a)', 'Outros',
       'Inválido', 'Filho (a) / Menor sob guarda / Enteado (a)',
       '(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a)'], dtype=object)

In [16]:
df

,ano,mes,tipo_pensao,data_inicio_pensao,total,ano_inicio_pensao,decada_inicio_pensao
0,2024,4,Pessoa designada,2002-05-29,1,2002,2000
1,2024,4,Pai / Mae,1995-07-17,1,1995,1990
2,2024,4,Pai / Mae,1993-07-22,1,1993,1990
3,2024,4,Pessoa designada,2003-08-20,1,2003,2000
4,2024,4,Pai / Mae,2003-08-22,1,2003,2000
...,...,...,...,...,...,...,...
283365,2024,4,(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a),2002-02-09,1,2002,2000
283366,2024,4,(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a),2002-01-01,1,2002,2000
283367,2024,4,(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a),2003-07-29,1,2003,2000
283368,2024,4,(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a),2008-02-16,1,2008,2000


In [17]:
df['decada_inicio_pensao'] = df['decada_inicio_pensao'].astype(int)

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_9064\1021241887.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['decada_inicio_pensao'] = df['decada_inicio_pensao'].astype(int)


In [22]:
df[df['tipo_pensao']=='Inválido'] # Corresponde a 0.58% do total

,ano,mes,tipo_pensao,data_inicio_pensao,total,ano_inicio_pensao,decada_inicio_pensao
4549,2024,4,Inválido,2009-11-25,1,2009,2000
4550,2024,4,Inválido,2013-04-18,1,2013,2010
4551,2024,4,Inválido,1994-05-19,1,1994,1990
4552,2024,4,Inválido,2003-08-22,1,2003,2000
4553,2024,4,Inválido,2020-09-01,1,2020,2020
...,...,...,...,...,...,...,...
6950,2024,4,Inválido,2010-05-01,1,2010,2010
6951,2024,4,Inválido,2013-05-06,1,2013,2010
6952,2024,4,Inválido,2012-03-09,1,2012,2010
6953,2024,4,Inválido,2006-12-01,1,2006,2000


In [23]:
df = df.drop(df.index[df['tipo_pensao']=='Inválido'])

In [25]:
df['tipo_pensao'].unique()

array(['Pessoa designada', 'Pai / Mae', 'Irmao (a)', 'Neto (a)', 'Outros',
       'Filho (a) / Menor sob guarda / Enteado (a)',
       '(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a)'], dtype=object)

In [26]:
pensoes = {'(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a)':'Cônjuge ou ex-cônjuge',
'Filho (a) / Menor sob guarda / Enteado (a)':"Filhos, netos, enteados ou menores em tutela",
'Pai / Mae':'Pais',
'Outros':'Outros',
'Pessoa designada':'Pessoa designada',
'Irmao (a)':'Irmão (ã)',
'Neto (a)':'Filhos, netos, enteados ou menores em tutela"'}

In [27]:
def categorizando(x):
    if x in pensoes:
        return pensoes[x]


In [28]:
df['tipo_pensao_agrupado'] = df['tipo_pensao'].apply(categorizando)

## Tratamento final

In [30]:
df= df[['ano', 'mes', 'data_inicio_pensao', 'ano_inicio_pensao', 'decada_inicio_pensao', 'tipo_pensao','tipo_pensao_agrupado', 'total']]
df

,ano,mes,data_inicio_pensao,ano_inicio_pensao,decada_inicio_pensao,tipo_pensao,tipo_pensao_agrupado,total
0,2024,4,2002-05-29,2002,2000,Pessoa designada,Pessoa designada,1
1,2024,4,1995-07-17,1995,1990,Pai / Mae,Pais,1
2,2024,4,1993-07-22,1993,1990,Pai / Mae,Pais,1
3,2024,4,2003-08-20,2003,2000,Pessoa designada,Pessoa designada,1
4,2024,4,2003-08-22,2003,2000,Pai / Mae,Pais,1
...,...,...,...,...,...,...,...,...
283365,2024,4,2002-02-09,2002,2000,(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a),Cônjuge ou ex-cônjuge,1
283366,2024,4,2002-01-01,2002,2000,(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a),Cônjuge ou ex-cônjuge,1
283367,2024,4,2003-07-29,2003,2000,(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a),Cônjuge ou ex-cônjuge,1
283368,2024,4,2008-02-16,2008,2000,(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a),Cônjuge ou ex-cônjuge,1


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 280964 entries, 0 to 283369
Data columns (total 8 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   ano                   280964 non-null  Int64         
 1   mes                   280964 non-null  Int64         
 2   data_inicio_pensao    280964 non-null  datetime64[ns]
 3   ano_inicio_pensao     280964 non-null  int64         
 4   decada_inicio_pensao  280964 non-null  int32         
 5   tipo_pensao           280964 non-null  object        
 6   tipo_pensao_agrupado  280964 non-null  object        
 7   total                 280964 non-null  Int64         
dtypes: Int64(3), datetime64[ns](1), int32(1), int64(1), object(2)
memory usage: 19.0+ MB


In [33]:
client = bigquery.Client()
dataset_ref = client.dataset('perfil_remuneracao')

In [36]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referência da observação'),
        bigquery.SchemaField('mes','INTEGER',description='Mês de referência da observação'), 
        bigquery.SchemaField('data_inicio_pensao','DATETIME',description='Data de início da pensão'),
        bigquery.SchemaField('ano_inicio_pensao','INTEGER',description='Ano de início da pensão extraido da data'),
        bigquery.SchemaField('decada_inicio_pensao','INTEGER',description='Década de início da pensão'),
        bigquery.SchemaField('tipo_pensao','STRING',description='Tipo de pensão'),
        bigquery.SchemaField('tipo_pensao_agrupado','STRING',description='Categorização das pensões'),
        bigquery.SchemaField('total','INTEGER',description='Quantidade total da observação')
        ]

In [37]:
table_ref = dataset_ref.table('SIAPE_inicio_pensoes_total')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result() 


LoadJob<project=repositoriodedadosgpsp, location=US, id=71612051-03e9-498e-b1dd-d8add6846c11>